In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[ ]:


from selenium import webdriver  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.chrome.options import Options 
import time
from bs4 import BeautifulSoup
import pandas as pd
import csv
from random import randint
from selenium.webdriver.support.wait import WebDriverWait
 
url="https://www.adsgroup.org.uk/member-capabilities/ppe/personal-protective-equipment/"
link=url
chrome_options = Options()  
chrome_options.add_argument("--headless")  
driver =webdriver.Chrome(executable_path=r'C:\Users\DELL\.wdm\drivers\chromedriver\80.0.3987.106\win32\chromedriver.exe')
driver.get(url)
time.sleep(3)

cat="PPE"
country="European"
data_link=[]
while True:
    elem_list=driver.find_elements_by_class_name("result")
    for e in elem_list:
        try:
            h2_tag=e.find_element_by_tag_name("h2")
            a_tag=h2_tag.find_element_by_tag_name("a").get_attribute("href")
            data_link.append(a_tag)
            print(a_tag)
        except Exception as e:
            print("NO A TAG",e)
            continue
    try:
        time.sleep(2)
        pagination=driver.find_element_by_class_name("pagination")
        #print(pagination.text)
        ul_tag=pagination.find_element_by_tag_name("ul")
        a_tagg=ul_tag.find_elements_by_tag_name("li")[-1]
        c_elem=a_tagg.find_element_by_tag_name("a")
        a_text=c_elem.text
        print(a_text)
        #c_elem.click()
        u=c_elem.get_attribute("href")
        print(u)
        if a_text=="»":
            driver.get(u)
            time.sleep(3)
    except Exception as e:
        print("NO NEXT",e)
        break
head=["Name","Address","domain","source","category","place"]
#skipped_links=[]
with open('uk_directory_hospital.csv', mode='w+',encoding="UTF-8",newline="") as file:
    writer1 = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer1.writerow(head)
    for li in data_link:
        try:
            driver.get(li)
            name=None
            address=None
            domain=None
            time.sleep(5)
            name_block=driver.find_element_by_tag_name("header")
            nn=name_block.find_element_by_tag_name("h1").text
            if nn is not None:
                name=nn.strip() 
                
            detail_block=driver.find_element_by_id('sidebar1')
            sub_block=detail_block.find_element_by_class_name("widget-3")
            main_block=sub_block.find_element_by_class_name("textwidget")
            
            add_block=main_block.find_element_by_tag_name("p")
            if add_block is not None:
                address=(add_block.text).strip()
            
            phn_block=main_block.find_element_by_class_name("telephone-fax")
            phn_data=phn_block.text
            if "Tel:" in phn_data:
                ind1=phn_data.index("Tel:")
                phone=phn_data[ind1+3:ind1+19]
                
            dom_block=main_block.find_elements_by_tag_name("a")
            for d in dom_block:
                #print(d.text)
                #print(d.get_attribute("href"))
                if d.text=="VISIT WEBSITE":
                    domain=d.get_attribute("href")
                
            
            print("NAME--",name)
            print("DOMAIN",domain)
            print("ADDRESS",address)
            print("PHONE",phone)
            writer1.writerow([name,address,domain,phone,link,cat,country])
        except Exception as e:
            print("skipp",e)
            continue



